# The Battle of Neighborhoods
## Applied Data Science Capstone
### By Andreia Queiroz Correia
#### Fev/2019

### 1) Import Libraries

In [27]:
import pandas as pd

import json # library to handle JSON files

import folium # map rendering library

import requests

from sklearn.cluster import KMeans

import numpy as np # library to handle data in a vectorized manner

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

### 2) Initialize variables

In [28]:
vConsultarFoursquare = 0
vLoadPandas = 0
FileVenuesName = 'nearby_venues.csv'
count = 0
num_top_venues = 5
PriceColors = ['blue','green','orange','red']
kclusters = 4

### 3) Choose the research criteria

In [29]:
#Sample: '1','2','3','4'
aPrice = ['1','2','3','4']

# Sample: 'Pizza Place', 'Italian Restaurant', 'Coffee Shop', 'Deli / Bodega'
aCategory = ['Pizza Place', 'Italian Restaurant']

### 4) Load pandas dataframe neighborhoods and venues from NYC

In [30]:
nyc_neighborhoods = pd.read_csv('NY_neighborhoods.csv')

In [31]:
CLIENT_ID = 'IWVS4ARY5DTCZHE3223EWLMVEMP2FKIPO1Z4FT1QUSWUJEOS' # your Foursquare ID
CLIENT_SECRET = 'RLCMR5EKZT2NWZF0GDIK15STWZVFHZBSYPKXBOHWBY5LOTMK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [32]:
from pathlib import Path

def getNearbyVenuesFourSquare(city,name, lat, long):
           
    if vConsultarFoursquare == 1:
        
        for price in range(1,5,1):
            
            LIMIT = 1000
            radius = 500
            count = 0
            venues_list=[]

            cols = ['City','Neighborhood', 'NLat', 'NLong', 'Venue', 'VenueLatitude', 'VenueLongitude', 'VenueCategory','Price']

            nearby_venues = pd.DataFrame(columns=cols)     

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT,
                price)
            
            print(url)
            
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']

            venues_list.append([(
                city,
                name,          
                lat,
                lng,
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name'],
                price) for v in results])

            nearby_venues_temp = pd.DataFrame([item for venue_list in venues_list for item in venue_list], columns=cols)

            File = Path(FileVenuesName)
            if File.is_file():
                nearby_venues = pd.read_csv(FileVenuesName)
                nearby_venues = nearby_venues.drop(nearby_venues.columns[0], axis = 1)
                nearby_venues = nearby_venues.append(nearby_venues_temp,sort=False,ignore_index=True)

            nearby_venues.to_csv('nearby_venues.csv')
        

In [33]:
from pathlib import Path

def getNearbyVenues(city,name, lat, long):

    File = Path(FileVenuesName)
    if File.is_file():
        nearby_venues = pd.read_csv(FileVenuesName)
                
        nearby_venues2 = nearby_venues.loc[(round(nearby_venues['NLat'].astype('float'),10) == round(lat,10)) & 
                                           (round(nearby_venues['NLong'].astype('float'),10) == round(long,10)) ]
        
        if nearby_venues2.empty:                                
            getNearbyVenuesFourSquare(city,name, lat, long)  
        else:
            print('valor DS')
    else:        
        getNearbyVenuesFourSquare(city,name, lat, long)  
            
        

In [34]:
if vLoadPandas == 1:
    count = 0

    for name, lat, lng in zip(nyc_neighborhoods['Neighborhood'], nyc_neighborhoods['Latitude'], nyc_neighborhoods['Longitude']):
        count += 1

        if count < 306:

                getNearbyVenues(city='NYC',
                                name=name,
                                lat=lat, 
                                long=lng)
else:
    nearby_venues = pd.read_csv(FileVenuesName)
        

In [35]:
nearby_venues.shape

(7857, 10)

In [36]:
nearby_venues['Price'] = nearby_venues['Price'].astype('str')

In [37]:
nearby_venues['Price2'] = nearby_venues['Price'].map({'1': '$', '2': '$$', '3': '$$$', '4': '$$$$'})
nearby_venues.head(2)

,Unnamed: 0,City,Neighborhood,NLat,NLong,Venue,VenueLatitude,VenueLongitude,VenueCategory,Price,Price2
0,0,NYC,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant,2,$$
1,1,NYC,Co-op City,40.874294,-73.829939,Capri II Pizza,40.876374,-73.829940,Pizza Place,1,$


In [38]:
nearby_venues_count = nearby_venues[['Neighborhood','Venue']].groupby(['Neighborhood']).count()
nearby_venues_count.rename(columns={'Venue':'Count'}, inplace=True)
nearby_venues_count = nearby_venues_count.sort_values(by=['Count'], ascending=False).head(num_top_venues)
nearby_venues_count

,Count
Neighborhood,
Upper East Side,701
Chinatown,161
Murray Hill,154
Greenwich Village,151
Little Italy,142


In [39]:
nearby_max_venues = nearby_venues.loc[nearby_venues['Neighborhood'].isin(nearby_venues_count.index.values)]

In [44]:
nearby_venues_price_count = nearby_max_venues[['Price','Venue']].groupby(['Price']).count()
nearby_venues_price_count.sort_values(by='Price', ascending=False)
nearby_venues_price_count

,Venue
Price,
1,446
2,515
3,213
4,135


In [14]:
nearby_venues_category_count = nearby_max_venues[['Neighborhood','Price','Venue']].groupby(['Neighborhood','Price']).count()
nearby_venues_category_count.sort_values(by='Neighborhood', ascending=False)
nearby_venues_category_count

Venue
Neighborhood      Price       
Chinatown         1         83
                  2         62
                  3         15
                  4          1
Greenwich Village 1         43
                  2         67
                  3         35
                  4          6
Little Italy      1         55
                  2         60
                  3         26
                  4          1
Murray Hill       1         46
                  2         84
                  3         16
                  4          8
Upper East Side   1        219
                  2        242
                  3        121
                  4        119

In [15]:
# one hot encoding
nyc_onehot = pd.get_dummies(nearby_max_venues[['Neighborhood']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nyc_onehot['Price'] = nearby_max_venues['Price'] 

# move neighborhood column to the first column
fixed_columns = [nyc_onehot.columns[-1]] + list(nyc_onehot.columns[:-1])
nyc_onehot = nyc_onehot[fixed_columns]

In [16]:
nyc_grouped = nyc_onehot.groupby('Price').mean().reset_index()
nyc_grouped.head()

,Price,Chinatown,Greenwich Village,Little Italy,Murray Hill,Upper East Side
0,1,0.186099,0.096413,0.123318,0.103139,0.491031
1,2,0.120388,0.130097,0.116505,0.163107,0.469903
2,3,0.070423,0.164319,0.122066,0.075117,0.568075
3,4,0.007407,0.044444,0.007407,0.059259,0.881481


In [17]:
for hood in nyc_grouped['Price']:
    print("----"+hood+"----")
    temp = nyc_grouped[nyc_grouped['Price'] == hood].T.reset_index()
    temp.columns = ['Neighborhood','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----1----
        Neighborhood  freq
0    Upper East Side  0.49
1          Chinatown  0.19
2       Little Italy  0.12
3  Greenwich Village  0.10
4        Murray Hill  0.10


----2----
        Neighborhood  freq
0    Upper East Side  0.47
1        Murray Hill  0.16
2  Greenwich Village  0.13
3          Chinatown  0.12
4       Little Italy  0.12


----3----
        Neighborhood  freq
0    Upper East Side  0.57
1  Greenwich Village  0.16
2       Little Italy  0.12
3        Murray Hill  0.08
4          Chinatown  0.07


----4----
        Neighborhood  freq
0    Upper East Side  0.88
1        Murray Hill  0.06
2  Greenwich Village  0.04
3          Chinatown  0.01
4       Little Italy  0.01




In [18]:
def return_most_common_prices(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
indicators = ['st', 'nd', 'rd']

if (nyc_grouped.shape[1] -1) < num_top_venues:
    num_top_venues = nyc_grouped.shape[1] -1

# create columns according to number of top venues
columns = ['Price']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Neighborhood'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Neighborhood'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Price'] = nyc_grouped['Price']

for ind in np.arange(nyc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_prices(nyc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Price,1st Neighborhood,2nd Neighborhood,3rd Neighborhood,4th Neighborhood,5th Neighborhood
0,1,Upper East Side,Chinatown,Little Italy,Murray Hill,Greenwich Village
1,2,Upper East Side,Murray Hill,Greenwich Village,Chinatown,Little Italy
2,3,Upper East Side,Greenwich Village,Little Italy,Murray Hill,Chinatown
3,4,Upper East Side,Murray Hill,Greenwich Village,Little Italy,Chinatown


In [20]:
nyc_grouped_clustering = nyc_grouped.drop('Price', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

nyc_grouped_clustering.head()

,Chinatown,Greenwich Village,Little Italy,Murray Hill,Upper East Side
0,0.186099,0.096413,0.123318,0.103139,0.491031
1,0.120388,0.130097,0.116505,0.163107,0.469903
2,0.070423,0.164319,0.122066,0.075117,0.568075
3,0.007407,0.044444,0.007407,0.059259,0.881481


In [21]:
nyc_merged = pd.DataFrame(neighborhoods_venues_sorted['Price'])

# merge nyc_grouped with NYC_data to add latitude/longitude for each neighborhood
nyc_merged = nyc_merged.join(neighborhoods_venues_sorted.set_index('Price'), on='Price')

In [22]:
nyc_merged.head()

,Price,1st Neighborhood,2nd Neighborhood,3rd Neighborhood,4th Neighborhood,5th Neighborhood
0,1,Upper East Side,Chinatown,Little Italy,Murray Hill,Greenwich Village
1,2,Upper East Side,Murray Hill,Greenwich Village,Chinatown,Little Italy
2,3,Upper East Side,Greenwich Village,Little Italy,Murray Hill,Chinatown
3,4,Upper East Side,Murray Hill,Greenwich Village,Little Italy,Chinatown


In [23]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [29]:
aNeighborhood[0][0]

'Upper East Side'

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

if nearby_venues.empty == False:
      
    aNeighborhood = neighborhoods_venues_sorted.drop('Price', axis=1).values.tolist()
    
    aNeighborhoodlist = list();
    aNeighborhoodlist.append(aNeighborhood[0][0])
        
    nearby_venues_selected  = nearby_venues.loc[nearby_venues['Neighborhood'].isin(aNeighborhoodlist)] 
        
    if nearby_venues.empty == False:

        x = np.arange(kclusters)
        ys = [i + x + (i*x)**2 for i in range(5)]
        colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
        rainbow = [colors.rgb2hex(i) for i in colors_array]

        for lat, long, venue, price, price2 in zip(nearby_venues_selected['VenueLatitude'], nearby_venues_selected['VenueLongitude'], nearby_venues_selected['Venue'], nearby_venues_selected['Price'], nearby_venues_selected['Price2']):
            label = folium.Popup(venue + ' - ' + price2, parse_html=True)
            folium.CircleMarker(
                    [lat, long],
                    radius=5,
                    popup=label,
                    color=PriceColors[int(price)-1],
                    fill=True,
                    fill_color=PriceColors[int(price)-1],
                    fill_opacity=0.7).add_to(map_clusters)

map_clusters
